In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("gamma_funds_20250611.csv")

In [4]:
df.shape

(18737, 27)

In [7]:
df["is_universo_investibile"].unique()

array([0, 1])

In [9]:
df_filtered = df[df["is_universo_investibile"]==1]
df_filtered.shape

(542, 27)

In [12]:
df_filtered["asset_class_to_report"].unique()

array(['Azionarie', 'Bilanciate', 'Alternative', 'Obbligazionarie',
       'Private Debt', 'Private Equity', 'not_available'], dtype=object)

In [10]:
df_filtered.columns

Index(['Unnamed: 0', 'codiceProdotto_frontoffice', 'ISIN',
       'nomeProdotto_frontoffice', 'divisa', 'tipoProdotto_frontoffice',
       'Tipologia Prodotto', 'fund_code', 'Asset Class', 'income',
       'multimanager', 'growth', 'lifestyle', 'ig_planning', 'protection',
       'retirement', 'active_management', 'tax_optimization', 'esg',
       'is_private_markets', 'expected_return', 'expected_volatility',
       'minimum_lot', 'risk_level', 'asset_class_to_report',
       'tipo_prodotto_to_report', 'is_universo_investibile'],
      dtype='object')

In [ ]:
[
    "Tipologia Prodotto", "Asset Class", 'risk_level', 'income',
    'multimanager', 'growth', 'lifestyle', 'ig_planning', 'protection',
    'retirement', 'active_management', 'tax_optimization', 'esg'
]

In [ ]:
import pandas as pd
import yfinance as yf

def compute_historic_metrics(price_series: list, moving_averages: list = [50, 200]) -> dict:
    """
    Compute key performance and risk metrics from a price series.

    Parameters:
    -----------
    price_series : list
        List of historical prices (daily).
    moving_averages : list
        List of periods for moving averages. Defaults to [50, 200].

    Returns:
    --------
    dict
        Dictionary of calculated metrics.
    """
    prices = pd.Series(price_series)
    returns = prices.pct_change().dropna()

    # Metrics
    cumulative_return = prices.iloc[-1] / prices.iloc[0] - 1
    annualized_return = (1 + cumulative_return) ** (252 / len(prices)) - 1
    volatility = returns.std() * np.sqrt(252)
    max_drawdown = ((prices / prices.cummax()) - 1).min()

    # Moving averages
    # ma_dict = {f"ma_{period}": prices.rolling(period).mean().iloc[-1] for period in moving_averages}

    return {
        'cumulative_return': cumulative_return,
        'annualized_return': annualized_return,
        'annualized_volatility': volatility,
        'max_drawdown': max_drawdown,
    }

def get_historical_cone(isin: str,
                        period: str = "5y",
                        interval: str = "1mo",
                        date_format: str = "%Y/%-m/%-d") -> dict:
    """
    Fetch daily historical price for a fund/security over a given period
    and return it in a JSON-like structure suitable for charting or API responses.

    Parameters:
    -----------
    isin : str
        The ISIN or ticker symbol of the fund/security.
    period : str
        Data period to download (e.g., '1y', '5y', 'max'). Defaults to '5y'.
    interval : str
        Data interval (e.g., '1d', '1wk', '1mo'). Defaults to '1d' for daily data.
    date_format : str
        Python strftime format for output dates. Default '%Y/%-m/%-d' (e.g. '2025/6/17').

    Returns:
    --------
    dict
        {
          'historical_cone': {
              'values': [...],            # list of float prices
              'time_period_dates': [...]  # list of formatted date strings
          }
        }
    """

    # Download data
    ticker = yf.Ticker(isin)
    hist = ticker.history(period=period, interval=interval)
    if hist.empty:
        raise ValueError(f"No historical data for ISIN/ticker: {isin}")

    # Use the 'Close' column
    series = hist['Close'].dropna()

    # Prepare output lists
    values = series.tolist()
    # Format dates; ensure month and day without leading zero where supported
    dates = [date.strftime(date_format) for date in series.index]

    metrics = compute_historic_metrics(values)

    return {"quantiles_data": {"real_data" : values}, "time_period_dates": dates, "statistics": metrics}



In [ ]:
output["historic_cone"] = get_historical_cone(isin=extracted_json["description"]["isin"])
